# Cost estimator

## Data cleanup

### Importing libraries

In [4]:
import numpy as np
import pandas as pd

In [8]:
crops = [
    "Apple",
    "Banana",
    "Blackgram",
    "Chickpea",
    "Coconut",
    "Coffee",
    "Cotton",
    "Grapes",
    "Jute",
    "Kidneybeans",
    "Lentil",
    "Maize",
    "Mango",
    "Mothbeans",
    "Mungbeans",
    "Orange",
    "Papaya",
    "Pigeonpeas",
    "Pomegranate",
    "Rice",
]
datasets = {}
for crop in crops:
    datasets[crop] = pd.read_csv(f"datasets/raw/crops/2023/{crop}.csv")
print(datasets)

{'Apple':              state district      market commodity   variety arrival_date  \
0            Bihar   Araria  Forbesganj     Apple  American   22/01/2023   
1            Bihar   Araria  Forbesganj     Apple  American   23/01/2023   
2            Bihar   Araria  Forbesganj     Apple  American   24/01/2023   
3            Bihar   Araria  Forbesganj     Apple  American   25/01/2023   
4            Bihar   Araria  Forbesganj     Apple  American   26/01/2023   
...            ...      ...         ...       ...       ...          ...   
70512  West Bengal  Kolkata      Mechua     Apple     Other   06/05/2023   
70513  West Bengal  Kolkata      Mechua     Apple     Other   15/06/2023   
70514  West Bengal  Kolkata      Mechua     Apple     Other   15/06/2023   
70515  West Bengal  Kolkata      Mechua     Apple     Other   30/10/2023   
70516  West Bengal  Kolkata      Mechua     Apple     Other   30/10/2023   

       min_price  max_price  modal_price update_date  
0         6000.0     9

In [117]:
location_data = pd.read_csv("datasets/raw/district_coords.csv")
def get_coords(state, district):
    # global count
    i = location_data[location_data["District"]==district]
    i = i[i["State"]==state.upper()]
    if i.shape[0] != 1:
        # with open("not_found.csv", "a") as f:
        #     f.write(f"{state},{district},{i.shape[0]}\n")
        # count += 1
        return (0,0)
    return (float(i["Latitude"].iloc[0]), float(i["Longitude"].iloc[0]))

In [59]:
print(get_coords(datasets["Apple"]["state"][4], datasets["Apple"]["district"][4]))

(26.137794, 87.464469)


In [60]:
print(datasets["Apple"])

             state district      market commodity   variety arrival_date  \
0            Bihar   Araria  Forbesganj     Apple  American   22/01/2023   
1            Bihar   Araria  Forbesganj     Apple  American   23/01/2023   
2            Bihar   Araria  Forbesganj     Apple  American   24/01/2023   
3            Bihar   Araria  Forbesganj     Apple  American   25/01/2023   
4            Bihar   Araria  Forbesganj     Apple  American   26/01/2023   
...            ...      ...         ...       ...       ...          ...   
70512  West Bengal  Kolkata      Mechua     Apple     Other   06/05/2023   
70513  West Bengal  Kolkata      Mechua     Apple     Other   15/06/2023   
70514  West Bengal  Kolkata      Mechua     Apple     Other   15/06/2023   
70515  West Bengal  Kolkata      Mechua     Apple     Other   30/10/2023   
70516  West Bengal  Kolkata      Mechua     Apple     Other   30/10/2023   

       min_price  max_price  modal_price update_date  
0         6000.0     9000.0     

Add latitude and longitude to all the datapoints

In [126]:
# count = 0
new_datasets = {}
for crop in crops:
    dataset = datasets[crop]
    coords = [get_coords(e["state"], e["district"]) for _,e in dataset.iterrows()]
    lats = [coord[0] for coord in coords]
    longs = [coord[1] for coord in coords]
    new_datasets[crop] = dataset.assign(latitude=lats).assign(longitude=longs).drop(["state","district","market","variety", "update_date"],axis=1)
    new_datasets[crop].to_csv(f"datasets/processed/crops/{crop}.csv", sep=",", index=False)
    # print(count)
print(new_datasets)

{'Apple':       commodity arrival_date  min_price  max_price  modal_price   latitude  \
0         Apple   22/01/2023     6000.0     9000.0       8500.0  26.137794   
1         Apple   23/01/2023     6500.0     8900.0       8500.0  26.137794   
2         Apple   24/01/2023     6400.0    10000.0       9000.0  26.137794   
3         Apple   25/01/2023     6200.0     9500.0       8500.0  26.137794   
4         Apple   26/01/2023     7000.0     9400.0       8200.0  26.137794   
...         ...          ...        ...        ...          ...        ...   
70512     Apple   06/05/2023     8000.0     9000.0       8400.0  22.548783   
70513     Apple   15/06/2023    11000.0    12000.0      11500.0  22.548783   
70514     Apple   15/06/2023     7000.0     8000.0       7500.0  22.548783   
70515     Apple   30/10/2023     9000.0    10000.0      10000.0  22.548783   
70516     Apple   30/10/2023     7000.0     8000.0       8000.0  22.548783   

       longitude  
0      87.464469  
1      87.46446